# 바로 실전이다.

실전을 통해 익혀보자. 

## 예제1

하나의 파일을 읽어 특정 문자 셋트를 찾아 바꾼 뒤에 새로운 파일을 만들어 보자. 

In [1]:
def main_ex01():

    replace_sets = [
        ( "尚", "尙" ),
        ( "為", "爲" ),
        ( "垒", "壘" ),
        ( "胆", "膽" )
    ]

    input_data_obj = open( "examples/example01.txt", 'r', encoding="utf-8" )
    input_data = input_data_obj.read()

    for ( before, after ) in replace_sets:
        input_data = input_data.replace( before, after )

    output_handler = open( "examples/example01RST.txt", 'w', encoding="utf-8")
    output_handler.write( input_data )
    output_handler.close()

main_ex01()

## 예제2

하나의 파일을 읽어 정해진 정규식으로 파일을 수정한 뒤에 새로운 파일에 저장해 보자.

In [10]:
def main_ex02():

    """
    \{([^\{]+?)[\=\:]\$(.+?)@(.+?)\}
    이북=교감기 변환
    \*$1\*\[\^$1\]★\[\^$1\]\: $1 \:*$2 ※$3★

    \{([^\{]+?)[\=\:](.+?)@(.+?)\}
    출처있는 주석
    \*$1\*\[\^$1\]★\[\^$1\]\: $1 \:$2 ※$3★

    \{([^\{^\}]+?)[\=\:](.+?)\}
    출처없는 주석
    \*$1\*\[\^$1\]★\[\^$1\]\: $1 \:$2★

    \{\:(.+?)\}
    대상없는 주석
    \[\^주석01\]★\[\^주석01\]\: \: $1★
    """

    import re   # https://docs.python.org/3/library/re.html

    patterns = [
        ( r"\{([^\{]+?)[\=\:]\$(.+?)@(.+?)\}", r"*\1*[^\1]★[^\1]: \1 :*\2 ※\3★" ), # 이북=교감기 변환
        ( r"\{([^\{]+?)[\=\:](.+?)@(.+?)\}", r"*\1*[^\1]★[^\1]: \1 :\2 ※\3★" ),    # 출처있는 주석
        ( r"\{([^\{^\}]+?)[\=\:](.+?)\}", r"*\1*[^\1]★[^\1]: \1 :\2★" ),            # 출처없는 주석
        ( r"\{\:(.+?)\}", r"\[\^주석01\]★[^주석01]: : \1★" ),                       # 대상없는 주석
    ]

    input_data_obj = open( "examples/example02.txt", 'r', encoding="utf-8" )
    input_data = input_data_obj.read()

    for ( before, after ) in patterns:
        input_data = re.sub( before, after, input_data )

    output_handler = open( "examples/example02RST.md", 'w', encoding="utf-8")
    output_handler.write( input_data )
    output_handler.close()
    
main_ex02()

## 예제3

주석을 따로 저장하여 별도의 파일을 만들어 보자. 

In [23]:
class Counter():
    
    def __init__(self):
        self.counter = 0
        
    def addOne(self):
        self.counter = self.counter + 1
        
    def puts(self):
        return "{:0>3}".format( self.counter )

        
class Refs():
    
    counter = Counter()
    refs = []
    
    @classmethod
    def appendNew(cls, ref_type, ref_body):
        cls.counter.addOne()
        ref_header = "[^{}{}]: ".format( ref_type, cls.counter.puts() )
        cls.refs.append( ref_header + ref_body )
        return ref_header

    @classmethod
    def callAll(cls):
        return cls.refs

    
def convertA( m ):  # https://docs.python.org/3/library/re.html#text-munging    
    matches = m.groups()
    ref_header = Refs.appendNew( 'A', "{} :*{} ※{}".format( *matches ) )
    return ref_header

def convertB( m ):  # https://docs.python.org/3/library/re.html#text-munging    
    matches = m.groups()
    ref_header = Refs.appendNew( 'B', "{} :{} ※{}".format( *matches ) )
    return ref_header

def convertC( m ):  # https://docs.python.org/3/library/re.html#text-munging    
    matches = m.groups()
    ref_header = Refs.appendNew( 'C', "{} :{} ".format( *matches ) )
    return ref_header

def convertD( m ):  # https://docs.python.org/3/library/re.html#text-munging    
    matches = m.groups()
    ref_header = Refs.appendNew( 'D', "{}".format( *matches ) )
    return ref_header

    
def main_ex03():

    import re   # https://docs.python.org/3/library/re.html

    patterns = [
        ( r"\{([^\{]+?)[\=\:]\$(.+?)@(.+?)\}", convertA ), # 이북=교감기 변환
        ( r"\{([^\{]+?)[\=\:](.+?)@(.+?)\}", convertB ),    # 출처있는 주석
        ( r"\{([^\{^\}]+?)[\=\:](.+?)\}", convertC ),            # 출처없는 주석
        ( r"\{\:(.+?)\}", convertD )                                 # 대상없는 주석
    ]

    input_data_obj = open( "examples/example02.txt", 'r', encoding="utf-8" )
    input_data = input_data_obj.read()

    for ( before, convertFunc ) in patterns:
        input_data = re.sub( before, convertFunc, input_data )

    output_handler1 = open( "examples/example03RST.md", 'w', encoding="utf-8")
    output_handler1.write( input_data )
    output_handler1.close()
    
    output_handler2 = open( "examples/example03RST_refs.md", 'w', encoding="utf-8")
    output_handler2.write( "\n".join( Refs.callAll() ) )
    output_handler2.close()
    
main_ex03()

## 예제4

주석을 따로 저장하여 별도의 파일을 만들어 보자. 2

In [31]:
class Refs2():
    
    counter = Counter()
    refs = []
    
    @classmethod
    def appendNew(cls, ref_type, ref_body):
        cls.counter.addOne()
        ref_header = "[^{}{}]: ".format( ref_type, cls.counter.puts() )
        cls.refs.append( ref_header + ref_body )
        return ref_header

    @classmethod
    def callAll( cls ):
        return cls.refs


def convertFunc( m ): 
    
    head, body_raw = m.groups()
    body = body_raw.replace("@", " ※").replace("$", "*")
    
    if head.strip() == "":
        ref_header = Refs2.appendNew( 'REF', body )
    else:
        ref_header = Refs2.appendNew( 'REF', head + " : " + body )
        
    return ref_header


def main_ex04():
   
    import re   # https://docs.python.org/3/library/re.html

    pattern = r"\{(.+?)[\:=](.+?)\}"

    input_data_obj = open( "examples/example02.txt", 'r', encoding="utf-8" )
    input_data = input_data_obj.read()

    input_data = re.sub( pattern, convertFunc, input_data )

    output_handler1 = open( "examples/example04RST.md", 'w', encoding="utf-8")
    output_handler1.write( input_data )
    output_handler1.close()
    
    output_handler2 = open( "examples/example04RST_refs.md", 'w', encoding="utf-8")
    output_handler2.write( "\n".join( Refs2.callAll() ) )
    output_handler2.close()
    
    
main_ex04()

## 예제5

하나의 폴더에 있는 여러가지 파일을 읽어 예제2에서 한 작업을 반복해 보자. 


In [11]:
import os

def allfiles( path, abs=False ):
    res = []

    for root, dirs, files in os.walk( path ):
        rootpath = os.path.join(os.path.abspath(path), root)
        if( abs ):
            filepath = [ os.path.join(rootpath, file) for file in files ]
        else:
            filepath = files
        res.append( filepath )

    return res


def main_ex05():

    target_dir = "./examples"
    print( allfiles( target_dir ) )

main_ex05()

[['example01.txt', 'example01RST.txt', 'example02.txt', 'example02RST.md']]
